In [1]:
!pip install -q datasets transformers nltk

In [2]:
from datasets import load_dataset

bst = load_dataset("blended_skill_talk", split="train")

print(bst)
print(bst[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['personas', 'additional_context', 'previous_utterance', 'context', 'free_messages', 'guided_messages', 'suggestions', 'guided_chosen_suggestions', 'label_candidates'],
    num_rows: 4819
})
{'personas': ["i've 2 kids.", 'i love flowers.'], 'additional_context': '', 'previous_utterance': ["I love live music, that's why I try to go to concerts", 'I do too. Wat do you like?'], 'context': 'empathetic_dialogues', 'free_messages': ['I like acting, I hope to be an actor, what about you?', 'No, but someday.', 'After I am done with school I plan to have a family.', 'I hope so, how old are your kids?', 'I would imagine. I am sure they a great kids.', 'I wish I had more time to do stuff like that. Medical school is exhausting. '], 'guided_messages': ['that is ok.  have any kids?', 'that is good. I have 2', 'that is great! you will be ready', '5 & 7.  they take up a lot of my time', 'luckily, they love flowers just as much as I do.  we spend a lot of time in the garden', '

In [3]:
from datasets import load_dataset

ultrachat = load_dataset(
    "HuggingFaceH4/ultrachat_200k",
    split="train_sft"
)

print(ultrachat)
print(ultrachat[0])


Dataset({
    features: ['prompt', 'prompt_id', 'messages'],
    num_rows: 207865
})
{'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?", 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57', 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections

In [4]:
from datasets import load_dataset

dolly = load_dataset(
    "databricks/databricks-dolly-15k",
    split="train"
)

print(dolly[0])


{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [5]:
from datasets import load_dataset

food = load_dataset("food101", split="train")

print(food[0])


{'image': <PIL.Image.Image image mode=RGB size=384x512 at 0x7A0D933D4B00>, 'label': 6}


In [ ]:
import random
import os
from datasets import load_dataset


BASE_PATH = "/content/data"
os.makedirs(BASE_PATH, exist_ok=True)

# How many samples from each dataset
N_ULTRACHAT = 20000
N_DOLLY = 8000
N_FOOD = 8000

random.seed(42)

ultrachat = load_dataset(
    "HuggingFaceH4/ultrachat_200k",
    split="train_sft"
)

ultra_text = []
for ex in ultrachat:
    convo = " ".join([m["content"] for m in ex["messages"]])
    ultra_text.append(convo)

ultra_text = random.sample(ultra_text, N_ULTRACHAT)
print("UltraChat samples:", len(ultra_text))


dolly = load_dataset(
    "databricks/databricks-dolly-15k",
    split="train"
)

dolly_text = []
for ex in dolly:
    prompt = ex["instruction"]
    if ex["context"]:
        prompt += " " + ex["context"]
    answer = ex["response"]
    dolly_text.append(f"Q: {prompt} A: {answer}")

dolly_text = random.sample(dolly_text, N_DOLLY)
print("Dolly samples:", len(dolly_text))


food = load_dataset("food101", split="train")

# Simple high-level nutrition descriptions
FAST_FOOD = {
    "pizza", "hamburger", "hot_dog", "french_fries",
    "fried_rice", "ice_cream", "donuts"
}

def food_to_text(label):
    name = label.replace("_", " ")
    if label in FAST_FOOD:
        return f"{name} is a popular fast food that is high in calories and should be eaten in moderation."
    else:
        return f"{name} is a type of food that can be part of a balanced diet."

food_text = []
for ex in food:
    label = food.features["label"].int2str(ex["label"])
    food_text.append(food_to_text(label))

food_text = random.sample(food_text, N_FOOD)
print("Food samples:", len(food_text))


all_text = ultra_text + dolly_text + food_text
random.shuffle(all_text)


split_idx = int(0.9 * len(all_text))
train_text = all_text[:split_idx]
val_text = all_text[split_idx:]


train_path = f"{BASE_PATH}/train.txt"
val_path = f"{BASE_PATH}/validation.txt"

with open(train_path, "w", encoding="utf-8") as f:
    for line in train_text:
        f.write(line.strip() + "\n")

with open(val_path, "w", encoding="utf-8") as f:
    for line in val_text:
        f.write(line.strip() + "\n")

print("\n✅ DATASET BUILD COMPLETE")
print("Train samples:", len(train_text))
print("Validation samples:", len(val_text))
print("Saved to:")
print(" -", train_path)
print(" -", val_path)


UltraChat samples: 20000
Dolly samples: 8000


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Food samples: 8000

✅ DATASET BUILD COMPLETE
Train samples: 32400
Validation samples: 3600
Saved to:
 - /content/data/train.txt
 - /content/data/validation.txt


In [7]:
!pip install -q transformers datasets matplotlib nltk tqdm

In [ ]:

import os
import time
import math
import zipfile
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    TrainerCallback
)
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download("punkt")


MODEL_NAME = "distilgpt2"
DATA_DIR = "/content/data"
OUTPUT_DIR = "/content/distilgpt2_core"
ZIP_PATH = "/content/distilgpt2_core_results.zip"

EPOCHS = 3
BATCH_SIZE = 8
MAX_LEN = 256
LR = 5e-5

GPU_POWER_W = 70
CPU_POWER_W = 15

os.makedirs(OUTPUT_DIR, exist_ok=True)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


dataset = load_dataset(
    "text",
    data_files={
        "train": f"{DATA_DIR}/train.txt",
        "validation": f"{DATA_DIR}/validation.txt"
    }
)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


class EpochPrinterCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n🚀 Starting Epoch {int(state.epoch) + 1} / {EPOCHS}")

    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"✅ Finished Epoch {int(state.epoch)}")


args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    logging_steps=200,
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=collator,
    callbacks=[EpochPrinterCallback()]
)


print("🔥 Training started...")
start_time = time.time()
trainer.train()
end_time = time.time()
print("🏁 Training finished")

runtime_sec = end_time - start_time


power_w = GPU_POWER_W if torch.cuda.is_available() else CPU_POWER_W
energy_joules = power_w * runtime_sec
energy_wh = energy_joules / 3600.0


trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Model weights saved")




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

✅ Finished Epoch 1

🚀 Starting Epoch 2 / 3
✅ Finished Epoch 2

🚀 Starting Epoch 3 / 3
✅ Finished Epoch 3


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


🏁 Training finished
✅ Model weights saved


Computing BLEU-2:   0%|          | 0/100 [00:00<?, ?it/s]

KeyError: 'text'

In [ ]:


import os
import time
import math
import zipfile
import random
import torch
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)




logs = trainer.state.log_history
train_losses = [x["loss"] for x in logs if "loss" in x]
eval_losses = [x["eval_loss"] for x in logs if "eval_loss" in x]

final_ppl = math.exp(eval_losses[-1])


smooth = SmoothingFunction().method1

def compute_bleu_2(model, tokenizer, raw_dataset, n_samples=50):
    scores = []
    model.eval()

    used = 0

    for i in tqdm(range(len(raw_dataset)), desc="Computing BLEU-2"):
        if used >= n_samples:
            break

        text = raw_dataset[i]["text"].strip()

        # 🔒 CRITICAL SAFETY CHECK
        if len(text.split()) < 3:
            continue

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=128
        )

        # Another safety check
        if inputs["input_ids"].shape[1] == 0:
            continue

        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=False
            )

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

        bleu = sentence_bleu(
            [text.split()],
            pred.split(),
            weights=(0.5, 0.5),
            smoothing_function=SmoothingFunction().method1
        )

        scores.append(bleu)
        used += 1

    return sum(scores) / len(scores)


bleu_2 = compute_bleu_2(model, tokenizer, raw_val)


plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(eval_losses, label="Validation Loss")
plt.xlabel("Training Steps / Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training Curve")
plt.savefig(f"{OUTPUT_DIR}/loss_curve.png")
plt.close()


with open(f"{OUTPUT_DIR}/metrics.txt", "w") as f:
    f.write("FOOD-ONLY CORE MODEL\n")
    f.write(f"Epochs: {EPOCHS}\n")
    f.write(f"Final validation loss: {eval_losses[-1]}\n")
    f.write(f"Perplexity: {final_ppl}\n")
    f.write(f"BLEU-2: {bleu_2}\n")
    f.write(f"Runtime (seconds): {runtime_sec}\n")
    f.write(f"Estimated energy (Wh): {energy_wh}\n")
    f.write(f"Energy proxy power (W): {power_w}\n")

print("✅ Metrics saved")


with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as z:
    for root, _, files in os.walk(OUTPUT_DIR):
        for f in files:
            if f.endswith((".bin", ".safetensors", ".json", ".png", ".txt")):
                full = os.path.join(root, f)
                z.write(full, arcname=os.path.relpath(full, OUTPUT_DIR))

print("📦 ZIP ready:", ZIP_PATH)

=
from google.colab import files
files.download(ZIP_PATH)


Computing BLEU-2:   0%|          | 0/84991 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

✅ Metrics saved
📦 ZIP ready: /content/distilgpt2_food_core.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>